# Introduction to Supervised Fine-Tuning

**Teaching models to follow instructions through demonstration**

## What is SFT?

**Supervised Fine-Tuning (SFT)** is the first step in post-training. It teaches a pre-trained model to follow instructions by training on (instruction, response) pairs.

The key insight is simple: if you want a model to answer questions, show it examples of good answers.

$$\mathcal{L}_{\text{SFT}} = -\sum_{t} \log P(y_t | x, y_{<t})$$

where:
- $x$ is the instruction/prompt
- $y$ is the response
- We maximize the probability of each response token given the instruction and previous tokens

## SFT vs Pre-Training

| Aspect | Pre-Training | SFT |
|--------|-------------|-----|
| **Data** | Raw text (books, web) | (instruction, response) pairs |
| **Objective** | Predict next token | Generate good responses |
| **Scale** | Trillions of tokens | Thousands-millions of examples |
| **Duration** | Weeks-months | Hours-days |
| **Learning rate** | Higher (1e-4) | Lower (1e-5 to 3e-4) |

## Popular SFT Datasets

### Alpaca (Stanford)
- **Size:** 52K instructions
- **Source:** GPT-3.5 generated from seed tasks
- **Format:** Instruction + optional input → output

### Dolly (Databricks)
- **Size:** 15K instructions
- **Source:** Human-written by Databricks employees
- **Quality:** Higher quality, more diverse

### OpenAssistant
- **Size:** 161K messages
- **Source:** Community-contributed conversations
- **Format:** Multi-turn conversations with rankings

In [1]:
from datasets import load_dataset

# Load the Alpaca dataset (cleaned version)
dataset = load_dataset("yahma/alpaca-cleaned", split="train")

print(f"Dataset size: {len(dataset)} examples")
print(f"Columns: {dataset.column_names}")
print()
print("Example:")
print(f"  Instruction: {dataset[0]['instruction']}")
if dataset[0]['input']:
    print(f"  Input: {dataset[0]['input']}")
print(f"  Output: {dataset[0]['output'][:100]}...")

README.md: 0.00B [00:00, ?B/s]

alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Dataset size: 51760 examples
Columns: ['output', 'input', 'instruction']

Example:
  Instruction: Give three tips for staying healthy.
  Output: 1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and...


## The SFT Training Loop

SFT follows a standard supervised learning pipeline:

```
1. Load pre-trained model
2. Format instruction data with chat template
3. Tokenize (instruction + response)
4. Apply loss masking (only compute loss on response tokens)
5. Train with standard cross-entropy loss
6. Save fine-tuned model
```

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load a small model for demonstration
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set pad token (GPT-2 doesn't have one by default)
tokenizer.pad_token = tokenizer.eos_token

print(f"Model: {model_name}")
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Vocabulary size: {tokenizer.vocab_size}")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model: gpt2
Parameters: 124,439,808
Vocabulary size: 50257


## Key SFT Concepts

The following notebooks cover these essential topics:

1. **Instruction Formatting** — How to structure prompts with chat templates
2. **Loss Masking** — Why we only compute loss on response tokens
3. **Training Loop** — Complete implementation with best practices
4. **LoRA** — Efficient fine-tuning with low-rank adaptation

Let's start with instruction formatting!